In [1]:
import pandas as pd
import numpy as np

### 1. 데이터 불러오기

In [2]:
final_df = pd.read_csv("./final1.csv", encoding='cp949')

final_df['등록일시'] = pd.to_datetime(final_df['등록일시'])
final_df.drop(['Unnamed: 0'], axis=1, inplace=True)
final_df.head()

,자전거번호,등록일시,이용시간,이용거리,고장구분
0,SPB-00003,2020-04-01,3,360,0
1,SPB-00003,2020-04-01,22,1961,0
2,SPB-00003,2020-04-01,38,3273,0
3,SPB-00003,2020-04-01,19,1696,0
4,SPB-00003,2020-04-01,6,609,0


In [3]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54044163 entries, 0 to 54044162
Data columns (total 5 columns):
 #   Column  Dtype         
---  ------  -----         
 0   자전거번호   object        
 1   등록일시    datetime64[ns]
 2   이용시간    int64         
 3   이용거리    int64         
 4   고장구분    int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 2.0+ GB


### 2. 누적이용시간, 누적이동거리, 누적이용횟수

In [4]:
#누적 이용시간 & 누적 이동거리, 이용횟수 빈 열 생성
final_df.loc[:, '누적이용시간'] = np.nan
final_df.loc[:, '누적이용거리'] = np.nan
final_df.loc[:, '누적이용횟수'] = np.nan

#이용시간과 이동거리를 계속 합하다가 고장구분에 1을 만나면 기록
#자전거 넘버가 바뀌면 0으로 리셋, 다시시작

final_df['누적이용시간'] = final_df.groupby(by=['자전거번호'])['이용시간'].apply(lambda x: x.cumsum())
final_df['누적이용거리'] = final_df.groupby(by=['자전거번호'])['이용거리'].apply(lambda x: x.cumsum())
final_df['누적이용횟수'] = final_df.groupby(by=['자전거번호']).cumcount()+1

### 3. 이용강도, 평균이용거리

In [5]:
#이용강도 = 단위시간 당 평균적으로 얼마를 달렸는지
#평균이용거리 = 1회 당 평균적으로 얼마를 달렸는지
final_df['이용강도'] = final_df['누적이용거리'] / final_df['누적이용시간']
final_df['평균이용거리'] = final_df['누적이용거리'] / final_df['누적이용횟수']

#모두 정수화
final_df = final_df.astype({'이용강도' : int, '평균이용거리' : int}, errors='raise')

In [6]:
#필요없는 열 삭제
final_df.drop(['이용시간', '이용거리'], axis=1, inplace=True)
final_df.head()

,자전거번호,등록일시,고장구분,누적이용시간,누적이용거리,누적이용횟수,이용강도,평균이용거리
0,SPB-00003,2020-04-01,0,3,360,1,120,360
1,SPB-00003,2020-04-01,0,25,2321,2,92,1160
2,SPB-00003,2020-04-01,0,63,5594,3,88,1864
3,SPB-00003,2020-04-01,0,82,7290,4,88,1822
4,SPB-00003,2020-04-01,0,88,7899,5,89,1579


### 4. 자전거 나이

In [7]:
#자전거 수명 구하기

#자전거 번호 별 첫 대여일만 남긴 프레임
first_df = final_df.drop_duplicates(['자전거번호'], keep='first') #첫 대여일만 남기기
first_df = first_df.iloc[:, :2] #자전거번호랑 등록일시만 남기기
first_df.rename(columns={'등록일시': '첫 대여일'}, inplace=True) #열 이름 바꾸기

#두 데이터 합치기
final_df = final_df.merge(right=first_df, how='left', on='자전거번호', suffixes=('_',''))

#수명 구하기
final_df['나이'] = final_df['등록일시'] - final_df['첫 대여일']

final_df.tail()
#결측치 없음

,자전거번호,등록일시,고장구분,누적이용시간,누적이용거리,누적이용횟수,이용강도,평균이용거리,첫 대여일,나이
54044158,SPB-90005,2020-08-19,0,36,3340,3,92,1113,2020-06-25,55 days
54044159,SPB-90005,2020-08-19,0,57,5197,4,91,1299,2020-06-25,55 days
54044160,SPB-90005,2020-08-19,0,62,5723,5,92,1144,2020-06-25,55 days
54044161,SPB-90005,2020-08-20,0,64,5970,6,93,995,2020-06-25,56 days
54044162,SPB-90005,2020-08-20,0,109,9848,7,90,1406,2020-06-25,56 days


### 5. 샘플링

In [8]:
#연도별로 자르기
df_2020 = final_df[final_df['등록일시'].dt.year == 2020]
df_2021 = final_df[final_df['등록일시'].dt.year == 2021]

#연도별 고장데이터 개수로 샘플링
break_2020 = df_2020[df_2020['고장구분'] == 1] #2020년에 고장난 자전거 데이터프레임
alive_2020 = df_2020[df_2020['고장구분'] == 0] #2020년에 멀쩡한 자전거 데이터프레임
alive_2020 = alive_2020.sample(break_2020.shape[0]) #고장난 개수만큼 샘플링

break_2021 = df_2021[df_2021['고장구분'] == 1] #2021년에 고장난 자전거 데이터프레임
alive_2021 = df_2021[df_2021['고장구분'] == 0] #2021년에 멀쩡한 자전거 데이터프레임
alive_2021 = alive_2021.sample(break_2021.shape[0]) #고장난 개수만큼 샘플링

In [19]:
data = pd.concat([break_2020, break_2021, alive_2020, alive_2021], ignore_index=True)

In [20]:
data.head()

,자전거번호,등록일시,고장구분,누적이용시간,누적이용거리,누적이용횟수,이용강도,평균이용거리,첫 대여일,나이
0,SPB-00004,2020-03-11,1,1160,134182,36,115,3727,2020-02-21,19 days
1,SPB-00004,2020-03-18,1,1921,202745,61,105,3323,2020-02-21,26 days
2,SPB-00004,2020-03-21,1,2117,222769,64,105,3480,2020-02-21,29 days
3,SPB-00004,2020-03-21,1,2216,231589,69,104,3356,2020-02-21,29 days
4,SPB-00004,2020-03-24,1,2841,293048,81,103,3617,2020-02-21,32 days


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367102 entries, 0 to 367101
Data columns (total 10 columns):
 #   Column  Non-Null Count   Dtype          
---  ------  --------------   -----          
 0   자전거번호   367102 non-null  object         
 1   등록일시    367102 non-null  datetime64[ns] 
 2   고장구분    367102 non-null  int64          
 3   누적이용시간  367102 non-null  int64          
 4   누적이용거리  367102 non-null  int64          
 5   누적이용횟수  367102 non-null  int64          
 6   이용강도    367102 non-null  int32          
 7   평균이용거리  367102 non-null  int32          
 8   첫 대여일   367102 non-null  datetime64[ns] 
 9   나이      367102 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), int32(2), int64(4), object(1), timedelta64[ns](1)
memory usage: 25.2+ MB


### 6.여름횟수

In [12]:
summer_df = data.loc[:, ['등록일시', '첫 대여일']]
summer_df

,등록일시,첫 대여일
0,2020-03-11,2020-02-21
1,2020-03-18,2020-02-21
2,2020-03-21,2020-02-21
3,2020-03-21,2020-02-21
4,2020-03-24,2020-02-21
...,...,...
367097,2021-07-22,2020-11-07
367098,2021-05-03,2020-11-04
367099,2021-09-25,2020-11-04
367100,2021-05-13,2020-06-03


In [13]:
#여름 횟수 구하는 함수
def get_summer(i) :
    #첫대여일~등록일시 까지의 월
    k = pd.period_range(start=summer_df.iloc[i, 1], end=summer_df.iloc[i, 0], freq='M')

    #프레임 만들어주기
    kk = pd.DataFrame(index=k, data=[0]*len(k))

    #월이 7월인 것만 빼오기
    kkk = list(filter(lambda x: x==7, kk.index.month))

    #7월의 개수 세기
    return(len(kkk))

In [22]:
index_list = data.index.to_series() #인덱스만 담은 시리즈

#df에 '여름'이라는 열을 만들고 여름 개수를 넣자
data['여름'] = index_list.map(lambda x: get_summer(x)) 

data

,자전거번호,등록일시,고장구분,누적이용시간,누적이용거리,누적이용횟수,이용강도,평균이용거리,첫 대여일,나이,여름
0,SPB-00004,2020-03-11,1,1160,134182,36,115,3727,2020-02-21,19 days,0
1,SPB-00004,2020-03-18,1,1921,202745,61,105,3323,2020-02-21,26 days,0
2,SPB-00004,2020-03-21,1,2117,222769,64,105,3480,2020-02-21,29 days,0
3,SPB-00004,2020-03-21,1,2216,231589,69,104,3356,2020-02-21,29 days,0
4,SPB-00004,2020-03-24,1,2841,293048,81,103,3617,2020-02-21,32 days,0
...,...,...,...,...,...,...,...,...,...,...,...
367097,SPB-52238,2021-07-22,0,27650,3324304,1047,120,3175,2020-11-07,257 days,1
367098,SPB-52345,2021-05-03,0,24829,2847288,931,114,3058,2020-11-04,180 days,0
367099,SPB-52487,2021-09-25,0,56592,6524948,2082,115,3133,2020-11-04,325 days,1
367100,SPB-37595,2021-05-13,0,18656,1947426,587,104,3317,2020-06-03,344 days,1


In [25]:
#필요없는 데이터 지우기
data.drop(['자전거번호', '등록일시', '누적이용시간', '누적이용거리', '누적이용횟수', '첫 대여일'], axis=1, inplace=True)

#나이열 str 형식으로 바꾸기
data = data.astype({'나이' : str}, errors='raise')

#나이열 숫자만 남기기
data['나이'] = data['나이'].str.split(' ').str[0]

#나이열 int 형식으로 바꾸기
data = data.astype({'나이' : int}, errors='raise')

### 7. 데이터셋 완성

In [30]:
#열이름 바꾸기
data.rename(columns={'고장구분': 'breakdown', '이용강도': 'intensity', '평균이용거리' : 'meanDist', '나이' : 'age', '여름' : 'summer'}, inplace=True)

In [31]:
data.head()

,breakdown,intensity,meanDist,age,summer
0,1,115,3727,19,0
1,1,105,3323,26,0
2,1,105,3480,29,0
3,1,104,3356,29,0
4,1,103,3617,32,0


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367102 entries, 0 to 367101
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   breakdown  367102 non-null  int64
 1   intensity  367102 non-null  int32
 2   meanDist   367102 non-null  int32
 3   age        367102 non-null  int32
 4   summer     367102 non-null  int64
dtypes: int32(3), int64(2)
memory usage: 9.8 MB


In [34]:
#데이터 저장
data.to_csv("bike_data.csv", mode='w')